In [11]:
from pathlib import Path
from glob import glob

import pandas as pd
import numpy as np

from ase import io, Atoms, Atom
from pmutt.statmech import StatMech, trans, vib, rot, elec

# interactive plots
%matplotlib widget

## Sorted according SO-ZORA

In [21]:
working_directory = Path("/home/danian/hg2w/calculaT/ThermoChemistry/structures")

In [23]:
list_file_adf = glob(str(working_directory / 'sozora') + '/*.out')
energies_file_adf = working_directory / ('sozora/energies.txt')

## Reading Coordinates and Frequencies from ADF.2013

In [24]:
find_lines_adf = {}
for file_adf in list_file_adf:
    read_file_adf = Path(file_adf).read_text()

    """ Lines to read the geometry in ADF.2013 """
    start_line_geom = None
    last_line_geom = None
    start_line_freq = None
    last_line_freq = None
    for count, line in enumerate(read_file_adf.split('\n')):

        if line == "                                            *  F R E Q U E N C I E S  *":
            start_line_geom = count + 11
        elif line == " Atomic Masses:":
            last_line_geom = count - 4
        elif line == " List of All Frequencies:":
            start_line_freq = count + 9
        elif line == " Statistical Thermal Analysis  ***  ideal gas assumed  ***":
            last_line_freq = count - 4
        elif start_line_geom and last_line_geom and start_line_freq and last_line_freq:

            break
    find_lines_adf[Path(file_adf).name] = [start_line_geom, last_line_geom, start_line_freq, last_line_freq]


In [77]:
geoms_systems = {}
freqs_systems = {}
energies_systems = {}
for file_adf in list_file_adf:
    with open(Path(file_adf), 'r') as f_adf:
        file_content = f_adf.readlines()

        """Atoms object with ase.Atoms"""
        last_line_geom = find_lines_adf[Path(file_adf).name][1]
        start_line_geom = find_lines_adf[Path(file_adf).name][0]
        lines_geom = last_line_geom - start_line_geom
        coord_system = []
        for i in range(lines_geom):
            line = file_content[start_line_geom + i].split()
            coord_system.append(Atom(line[1], (line[5], line[6], line[7])))
        geoms_systems[Path(file_adf).name] = Atoms(coord_system)

        """Frequencies"""
        last_line_freq = find_lines_adf[Path(file_adf).name][3]
        start_line_freq = find_lines_adf[Path(file_adf).name][2]
        lines_freq = last_line_freq - start_line_freq
        freq_system = []
        for i in range(lines_freq):
            line = file_content[start_line_freq + i].split()
            if float(line[0]) > 20.0:
                freq_system.append(float(line[0]))
        freqs_systems[Path(file_adf).name] = freq_system

    """Energies from ADF.2013 in Ev"""
    with open(energies_file_adf, 'r') as f_energies:
        stop = False
        while stop == False:
            line = f_energies.readline().split()
            if line[0] == Path(file_adf).with_suffix('').name:
                energies_systems[Path(file_adf).name] = float(line[1])
                stop = True

In [78]:
translation = {}
vibration = {}
rotation = {}
electronic = {}
for count, file_adf in enumerate(list_file_adf):
    name_file = Path(file_adf).name

    '''Translational'''
    translation[name_file] = trans.FreeTrans(n_degrees=3, atoms=geoms_systems[name_file])

    '''Vibrational'''
    vibration[name_file] = vib.HarmonicVib(vib_wavenumbers=freqs_systems[name_file]) #cm^-1

    '''Rotational'''
    rotation[name_file] = rot.RigidRotor(symmetrynumber=1, atoms=geoms_systems[name_file]) #simmetry point C1

    '''Electronic'''
    electronic[name_file] = elec.GroundStateElec(potentialenergy=float(energies_systems[name_file]), spin=0) #Ev


In [79]:
statmech = {}
for file_adf in list_file_adf:
    name_file = Path(file_adf).name
    name = Path(file_adf).with_suffix('').name
    '''StatMech Initialization'''
    statmech[name_file] = StatMech(name= name,
                            trans_model=translation[name_file],
                            vib_model=vibration[name_file],
                            rot_model=rotation[name_file],
                            elec_model=electronic[name_file])

In [80]:
print('systems     H[kcal/mol]    S[kcal/mol/K]    G[kcal/mol]     T: 298.15 K')
for file_adf in list_file_adf:
    name_file = Path(file_adf).name
    H_statmech = statmech[name_file].get_H(T=298.15, units='kcal/mol')
    S_statmech = statmech[name_file].get_S(T=298.15, units='kcal/mol/K')
    G_statmech = statmech[name_file].get_G(T=298.15, units='kcal/mol')
    print(f"{name_file}:  {H_statmech:.3f}      {S_statmech:.3f}       {G_statmech:.3f}")
    # print(f'H_w6s1(T=298.15) = {H_statmech:.3f} kcal/mol')
    # print(f'S_w6s1(T=298.15) = {S_statmech:.3f} kcal/mol/K')
    # print(f'G_w6s1(T=298.15) = {G_statmech:.3f} kcal/mol')

systems     H[kcal/mol]    S[kcal/mol/K]    G[kcal/mol]     T: 298.15 K
w5s10.out:  -26702.474      0.124       -26739.555
w5s9.out:  -26704.981      0.123       -26741.711
w6s10.out:  -27049.573      0.130       -27088.411
w5s6.out:  -26711.176      0.120       -26746.822
w6s9.out:  -27049.488      0.135       -27089.753
w6s7.out:  -27052.128      0.130       -27090.758
w5s7.out:  -26705.714      0.115       -26740.100
w2s1.out:  -25668.161      0.077       -25691.234
w6s2.out:  -27059.177      0.123       -27095.863
w4s4.out:  -26361.056      0.116       -26395.700
w6s18.out:  -27031.408      0.147       -27075.240
w3s2.out:  -26016.938      0.099       -26046.364
w6s11.out:  -27049.072      0.127       -27086.809
w6s19.out:  -27059.605      0.127       -27097.547
w6s5.out:  -27056.559      0.131       -27095.725
w5s2.out:  -26722.905      0.113       -26756.606
w5s3.out:  -26719.370      0.115       -26753.703
w5s8.out:  -26698.493      0.132       -26737.733
w6s17.out:  -27037.143 